# Spectral clustering

Using the spectral clustering, firstly，we should calcule the distance between two points.

In [ ]:
import numpy as np
from sklearn import preprocessing
import matplotlib.pyplot as plt
from sklearn.cluster import SpectralClustering

def distance_euclid(v1, v2):
    return np.sqrt(sum(np.power(v1 - v2, 2)))


Secondly, we use the data of each personne and let them as a tuple. Then we should normalize these datas. 

In [ ]:
dataset_all = np.loadtxt("phpOJxGL9.csv", delimiter=",", unpack=True, dtype=str)

dataset = dataset_all[:9, 1:]
dataset = dataset.astype(np.float)
dataset = dataset.T
dataset = preprocessing.MinMaxScaler().fit_transform(dataset)  # Normalisation
print(dataset)

[[0.70930233 1.         0.00402145 ... 0.00162635 0.5942029  0.52173913]
 [0.6744186  0.         0.14075067 ... 0.0182964  0.69565217 0.5       ]
 [0.6744186  0.         0.0924933  ... 0.01179101 0.62318841 0.52173913]
 ...
 [0.55813953 0.         0.00536193 ... 0.00792844 0.53623188 0.5       ]
 [0.31395349 0.         0.01206434 ... 0.00447245 0.5942029  0.54347826]
 [0.39534884 0.         0.0080429  ... 0.00284611 0.66666667 0.76086957]]


Then we should make a matrix which contains the distance among the points. The matirx is called Euclidian distance matrix.

In [ ]:
def matrix_distance_euclid(dataset):
    M = np.zeros((len(dataset), len(dataset)))
    for i in range(len(dataset)):
        for j in range(i + 1, len(dataset)):
            M[i][j] = distance_euclid(dataset[i], dataset[j])
            M[j][i] = M[i][j]
    return M

En suite, on peut obtenir un adjacency matrix W。Use the KNN algorithm to traverse all the sample points,and take the $k$ nearest points of each sample as the nearest neighbors,and only the k points closet to the sample are $W_{ij}=exp(-\frac{||x_{i}-x_{j}||^2}{(2\sigma)^2})$

In [ ]:
def matrix_adjacent(mde, dataset, k=53, sigma=0.15):  # KNN
    M = np.zeros((len(mde), len(mde)))
    for i in range((len(mde))):
        distance_i = zip(mde[i], range(len(mde)))
        distance_i = sorted(distance_i, key=lambda x: x[0])
        neighbors = [distance_i[m][1] for m in range(k + 1)]

        for j in neighbors:
            M[i][j] = np.exp(-sum(np.power(dataset[i] - dataset[j], 2)) / (2 * sigma ** 2))
            M[j][i] = M[i][j]
    return M

Après, on peut obtenir un matrix laplacien.
$ L=D-W $
Dans ce cas, on peut obtenir n valeurs propres qui sont positives.标准化后拉普拉斯矩阵：
$ D^{-1}LD^{-1} $

In [ ]:
def matrix_laplacian(ma):
    matrix_degree = np.diag(np.sum(ma, axis=0))

    ml = matrix_degree - ma

    # Normalisation
    ml_N = np.dot(np.dot(np.linalg.inv(matrix_degree) ** 0.5, ml), np.linalg.inv(matrix_degree) ** 0.5)
    # ml_N = np.dot(np.linalg.inv(matrix_degree), ml)
    return ml_N

In [ ]:
m, n = dataset.shape
mde = matrix_distance_euclid(dataset)
ma = matrix_adjacent(mde, dataset, 53, 0.15)
ml = matrix_laplacian(ma)
print(mde)
print(ma)
print(ml)

[[0.         1.08166674 1.03635717 ... 1.01381081 1.07578223 1.0776502 ]
 [1.08166674 0.         0.15370525 ... 0.42396743 0.53623823 0.53958514]
 [1.03635717 0.15370525 0.         ... 0.28829836 0.44056239 0.44727523]
 ...
 [1.01381081 0.42396743 0.28829836 ... 0.         0.25719282 0.33468017]
 [1.07578223 0.53623823 0.44056239 ... 0.25719282 0.         0.24396447]
 [1.0776502  0.53958514 0.44727523 ... 0.33468017 0.24396447 0.        ]]
[[1.         0.         0.         ... 0.         0.         0.        ]
 [0.         1.         0.59155139 ... 0.         0.         0.        ]
 [0.         0.59155139 1.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 1.         0.         0.        ]
 [0.         0.         0.         ... 0.         1.         0.        ]
 [0.         0.         0.         ... 0.         0.         1.        ]]
[[ 0.94644395  0.          0.         ...  0.          0.
   0.        ]
 [ 0.          0.85153075 -0.05437764 .

Calculer la matrice de contiguité W par sélectionner la fonction noyau gaussienne.

In [ ]:
num = []
eigenvalues, eigenvectors = np.linalg.eig(ml)
eigenvectors = np.abs(eigenvectors)
print(eigenvectors)

[[3.29217843e-03 2.86758390e-03 3.31408677e-03 ... 5.96503733e-16
  1.62226363e-14 1.19487669e-14]
 [9.71679464e-18 3.16536959e-17 4.52790392e-17 ... 1.27233744e-13
  6.10874929e-12 3.41836204e-12]
 [2.22683159e-17 1.35159163e-17 3.32455402e-18 ... 2.11099442e-12
  2.32164204e-11 1.40082091e-11]
 ...
 [8.19132248e-18 6.75862455e-17 6.88801739e-17 ... 2.96187692e-11
  3.12770518e-11 2.73110490e-12]
 [3.20623569e-17 6.37124694e-17 7.61741258e-17 ... 2.93918785e-10
  4.16664499e-10 3.13657288e-11]
 [2.69558459e-17 5.87603355e-17 7.30469256e-17 ... 9.30825013e-11
  1.23440454e-10 5.19217308e-12]]


Dans la fonction ici, on peut calculer les valeurs propres et les vecteurs propres de la matrice laplacienne

In [ ]:
from sklearn.cluster import KMeans
sp = KMeans(n_clusters=4).fit(eigenvectors)
kl = sp.labels_
cluster = np.unique(kl)
print(kl)

[3 2 0 0 0 0 3 3 0 0 0 0 0 3 0 0 0 0 3 3 0 0 0 0 0 1 1 1 2 1 0 0 0 3 3 0 3
 1 0 0 0 0 0 0 1 0 0 1 3 2 3 3 0 0 2 2 0 3 0 0 3 0 0 0 0 0 0 0 0 0 3 3 3 0
 0 3 0 1 0 1 1 0 0 0 0 0 0 0 0 0 0 0 2 0 0 0 0 2 0 0 2 0 0 3 0 0 0 0 0 0 3
 0 0 0 0 1 1 1 1 1 2 2 0 3 0 0 0 0 1 0 0 3 3 0 0 1 3 0 0 0 0 1 0 0 3 0 3 0
 0 0 0 0 3 0 0 0 0 0 0 0 2 2 0 0 0 0 1 3 1 2 0 3 2 0 0 0 0 1 2 2 2 2 2 0 0
 0 2 0 2 3 0 0 0 0 0 2 0 3 0 1 0 0 3 0 0 0 0 1 3 3 0 0 0 3 0 0 0 0 2 0 0 0
 0 0 0 0 0 3 3 0 0 0 0 1 3 0 0 3 3 0 0 0 3 1 0 0 2 0 0 0 0 0 0 3 3 2 2 0 0
 2 2 2 0 3 0 0 0 3 2 2 2 2 1 2 0 0 0 0 0 3 0 0 0 0 0 0 3 0 3 0 3 0 0 0 0 0
 3 3 3 3 0 3 3 0 3 3 0 0 3 3 0 1 0 3 0 0 0 0 0 0 3 3 0 0 0 0 3 0 0 0 0 0 0
 3 3 1 2 0 0 0 0 3 0 0 0 0 0 0 0 3 0 0 3 3 3 0 0 0 0 1 0 3 0 3 0 0 0 0 3 3
 1 0 2 3 3 0 0 3 3 0 0 0 3 3 0 0 0 0 0 0 0 0 0 0 3 0 0 0 0 0 3 2 3 0 0 3 0
 2 0 0 2 3 0 0 0 2 2 0 0 1 1 0 0 0 3 0 0 0 3 1 3 3 0 3 3 3 0 0 2 3 3 3 1 3
 0 0 0 1 3 3 0 0 2 0 0 3 0 0 2 2 3 3 0 0 3 3 3 3 0 0 0 3 0 0 0 0 0 0 2 2 0
 0 0 0 0 1 3 0 0 0 3 0 1 

Afficher à quel cluster chaque point appartient

In [ ]:
for i in cluster:
    num.append(np.sum(kl == i))
print(num)

[353, 46, 66, 118]
